In [10]:
import pandas as pd
import psycopg2
import re

def df_nan_filter(df):
    df = df[df["Size"].notnull()]              # remove "Size" row if size is null
    df["Languages"].fillna("EN", inplace=True) # set "EN" as langage if value is null
    df["Price"].fillna(0.0, inplace=True)      # set 0.0 as price if value is null
    df["Average User Rating"].fillna(df["Average User Rating"].median(), inplace=True) # set median as user_rating if value is null
    df["User Rating Count"].fillna(1, inplace=True) #replace nan counts with 1
    df.drop_duplicates(subset ="ID", inplace = True)
    return (df)

import ast

def string_filter(s: str):
    s = ast.literal_eval("b'''%s'''" % s)
    s = s.decode('raw_unicode_escape').encode('ascii', 'ignore')
    s = re.sub('[\t\n\r\v\f]', ' ', s.decode())
    s = re.sub(' +', ' ', s)
    s = s.strip('"')
    return (s)

df = pd.read_csv("appstore_games.csv")
df = df[["ID", "Name", "Average User Rating",
          "User Rating Count", "Price", "Description",
          "Developer", "Age Rating", "Languages",
          "Size", "Primary Genre", "Genres",
          "Original Release Date", "Current Version Release Date"]]

df = df_nan_filter(df)

for col in df:
    df[col] = df[col].apply(lambda x: string_filter(str(x)))

df.to_csv("filtered_appstore_games.csv")

/home/misteir/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [11]:
def delete_table():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute('DROP TABLE test;')          
    conn.commit()
    conn.close()

def create_table():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute("""CREATE TABLE test(
             Id bigint PRIMARY KEY,
             Name varchar,
             Avg_user_rating float,
             User_rating_count float,
             Price float,
             Description varchar,
             Developer varchar,
             Age_rating varchar,
             Languages varchar,
             Size float, 
             Primary_genre varchar,
             Genres varchar,
             Release_date date,
             Last_update date
             );""")           
    conn.commit()
    conn.close()

def populate(df):
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    for idx in range(df.shape[0]):
        tmp = df.iloc[idx]
        curr.execute("""INSERT INTO test
            (Id, Name, Avg_user_rating, User_rating_count, Price, Description, Developer,
            Age_rating, Languages, Size, Primary_genre, Genres, Release_date, Last_update
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                (tmp["ID"], tmp["Name"], tmp["Average User Rating"], tmp["User Rating Count"],
                 tmp["Price"], tmp["Description"], tmp["Developer"], tmp["Age Rating"],
                 tmp["Languages"], tmp["Size"], tmp["Primary Genre"], tmp["Genres"],
                 tmp["Current Version Release Date"], tmp["Current Version Release Date"]))
    conn.commit()
    conn.close()

def display_table():
    conn = psycopg2.connect(
        database="appstore_games",
        host="localhost",
        user="postgres_user",
        password="12345"
    )
    curr = conn.cursor()
    curr.execute("""SELECT * FROM test""")
    response = curr.fetchall()
    for row in response[:100]:
        print(row)
    conn.close()

In [12]:
delete_table()
create_table()

In [13]:
insert_table(df)

In [15]:
#display_table()